<a href="https://colab.research.google.com/github/CodeHunterOfficial/ABC_DataMining/blob/main/NLP/NLP-2025/%D0%A2%D0%BE%D0%BA%D0%B5%D0%BD%D0%B8%D0%B7%D0%B0%D1%82%D0%BE%D1%80%D1%8B%20(%D0%BF%D1%80%D0%B8%D0%BC%D0%B5%D1%80%D1%8B)_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BPE

In [ ]:
import collections

class BPE:
    """
    Реализация алгоритма Byte-Pair Encoding (BPE) для токенизации текста.
    """
    def __init__(self, vocab_size=None, num_merges=None):
        """
        Инициализирует BPE токенизатор.
        :param vocab_size: Желаемый максимальный размер словаря субтокенов.
                           Если указан, num_merges будет игнорироваться.
        :param num_merges: Количество итераций объединения пар.
                           Если vocab_size не указан, будет использоваться это значение.
        """
        if vocab_size is None and num_merges is None:
            raise ValueError("Необходимо указать либо vocab_size, либо num_merges.")

        self.vocab_size = vocab_size
        self.num_merges = num_merges
        self.merges = {}  # Словарь для хранения выполненных объединений {pair: new_token}
        self.vocabulary = set() # Текущий словарь субтокенов
        self.token_to_id = {} # Словарь для преобразования токенов в ID
        self.id_to_token = [] # Список для преобразования ID в токены

    def _get_initial_tokens(self, text):
        """
        Разбивает текст на начальные символьные токены и добавляет символ конца слова.
        :param text: Входная строка.
        :return: Список списков символьных токенов для каждого слова.
        """
        # Заменяем подчеркивания на пробелы для разделения слов, затем разбиваем по пробелам
        words = text.replace('_', ' ').split()
        initial_tokens = []
        for word in words:
            # Добавляем символ конца слова к каждому слову
            initial_tokens.append(list(word) + ['</w>'])
        return initial_tokens

    def _get_pair_counts(self, tokenized_corpus):
        """
        Подсчитывает частоту встречаемости всех смежных пар токенов в корпусе.
        :param tokenized_corpus: Список списков токенов (представляющих слова).
        :return: Словарь с частотами пар {('token1', 'token2'): count}.
        """
        pair_counts = collections.defaultdict(int)
        for word_tokens in tokenized_corpus:
            # Итерируем по парам токенов в каждом слове
            for i in range(len(word_tokens) - 1):
                pair_counts[(word_tokens[i], word_tokens[i+1])] += 1
        return pair_counts

    def _merge_pair(self, tokenized_corpus, pair, new_token):
        """
        Объединяет все вхождения заданной пары в новый токен в корпусе.
        :param tokenized_corpus: Текущий корпус токенов.
        :param pair: Пара токенов для объединения (tuple).
        :param new_token: Новый токен, который заменит пару.
        :return: Обновленный корпус токенов.
        """
        updated_corpus = []
        for word_tokens in tokenized_corpus:
            new_word_tokens = []
            i = 0
            while i < len(word_tokens):
                # Проверяем, является ли текущая позиция началом искомой пары
                if i + 1 < len(word_tokens) and (word_tokens[i], word_tokens[i+1]) == pair:
                    new_word_tokens.append(new_token)
                    i += 2 # Пропускаем оба токена, которые были объединены
                else:
                    new_word_tokens.append(word_tokens[i])
                    i += 1
            updated_corpus.append(new_word_tokens)
        return updated_corpus

    def fit(self, corpus_text):
        """
        Обучает BPE модель на заданном текстовом корпусе.
        :param corpus_text: Строка, представляющая обучающий корпус.
        """
        # Шаг 1: Инициализация
        # Начальное разбиение на символы и добавление символа конца слова
        current_corpus_tokens = self._get_initial_tokens(corpus_text)

        # Формирование начального словаря из всех уникальных символов
        for word_tokens in current_corpus_tokens:
            for token in word_tokens:
                self.vocabulary.add(token)

        # Определяем количество итераций объединения
        if self.vocab_size is not None:
            # Если задан размер словаря, вычисляем необходимое количество объединений
            # (целевой размер словаря - текущий размер словаря)
            num_iterations = self.vocab_size - len(self.vocabulary)
        elif self.num_merges is not None:
            num_iterations = self.num_merges
        else:
            # Это условие не должно быть достигнуто благодаря проверке в __init__
            num_iterations = 0

        # Шаг 2: Итеративное объединение
        for i in range(num_iterations):
            pair_counts = self._get_pair_counts(current_corpus_tokens)

            if not pair_counts: # Если нет пар для объединения, останавливаемся
                break

            # Находим наиболее частую пару
            # Сортируем по частоте (убывание), затем по лексикографическому порядку (возрастание)
            best_pair = max(pair_counts, key=lambda p: (pair_counts[p], -len(p[0]) - len(p[1]), p))

            # Если частота лучшей пары меньше 1, или нет пар для объединения, останавливаемся
            if pair_counts[best_pair] < 1:
                break

            # Формируем новый токен из объединенной пары
            new_token = "".join(best_pair)

            # Сохраняем объединение
            self.merges[best_pair] = new_token

            # Обновляем корпус, заменяя все вхождения пары новым токеном
            current_corpus_tokens = self._merge_pair(current_corpus_tokens, best_pair, new_token)

            # Добавляем новый токен в словарь
            self.vocabulary.add(new_token)

            # Вывод результатов каждого шага
            print(f"Итерация {i+1}: Объединение '{best_pair[0]}{best_pair[1]}' в '{new_token}'. Частота: {pair_counts[best_pair]}")
            print(f"Обновленный корпус: {current_corpus_tokens}")
            print(f"Текущий словарь: {sorted(list(self.vocabulary))}\n")

        # После обучения создаем отображения токенов в ID и обратно
        self.id_to_token = sorted(list(self.vocabulary))
        self.token_to_id = {token: i for i, token in enumerate(self.id_to_token)}

    def encode(self, text):
        """
        Токенизирует новую строку, используя обученные объединения.
        :param text: Строка для токенизации.
        :return: Список ID токенов.
        """
        # Начальное разбиение текста на символы с добавлением </w>
        words_tokens = self._get_initial_tokens(text)
        encoded_tokens_ids = []

        for word_tokens in words_tokens:
            # Для каждого слова применяем объединения в порядке их обучения
            # Создаем копию списка токенов слова для модификации
            current_word_tokens = list(word_tokens)

            # Применяем объединения итеративно, пока не сможем найти более длинный токен
            # или пока не закончатся объединения
            while True:
                merged_once = False
                new_word_tokens = []
                i = 0
                while i < len(current_word_tokens):
                    found_merge = False
                    # Ищем самое длинное возможное объединение, которое начинается с текущего токена
                    # Проходим по всем известным объединениям в порядке их создания (от коротких к длинным)
                    # или просто ищем самое длинное совпадение
                    best_match_len = 0
                    best_match_token = None

                    # Для простоты, будем применять объединения из self.merges
                    # в порядке их создания (от коротких к длинным)
                    # Это не совсем оптимально, но для демонстрации подходит
                    # В реальных реализациях используют более сложные структуры данных
                    # для эффективного поиска наиболее длинного совпадения
                    for (p1, p2), merged_token in self.merges.items():
                        if i + 1 < len(current_word_tokens) and \
                           current_word_tokens[i] == p1 and \
                           current_word_tokens[i+1] == p2:
                            # Проверяем, является ли это объединение частью более длинного
                            # уже существующего в словаре токена
                            if len(merged_token) > best_match_len:
                                best_match_len = len(merged_token)
                                best_match_token = merged_token
                                found_merge = True

                    if found_merge:
                        new_word_tokens.append(best_match_token)
                        i += 2
                        merged_once = True
                    else:
                        new_word_tokens.append(current_word_tokens[i])
                        i += 1

                if not merged_once:
                    break # Больше нет объединений для этого слова

                current_word_tokens = new_word_tokens

            # Преобразуем токены слова в их ID
            for token in current_word_tokens:
                if token in self.token_to_id:
                    encoded_tokens_ids.append(self.token_to_id[token])
                else:
                    # Если токен не найден (например, из-за OOV или неполного обучения),
                    # разбиваем его на символы и добавляем их ID
                    for char in token:
                        if char in self.token_to_id:
                            encoded_tokens_ids.append(self.token_to_id[char])
                        else:
                            # Этого не должно произойти, если начальный словарь содержит все символы
                            print(f"Предупреждение: Символ '{char}' не найден в словаре.")
                            # Можно добавить токен для неизвестных символов, например, <unk>
        return encoded_tokens_ids

    def decode(self, token_ids):
        """
        Декодирует последовательность ID токенов обратно в строку.
        :param token_ids: Список ID токенов.
        :return: Декодированная строка.
        """
        decoded_tokens = []
        for token_id in token_ids:
            if token_id < len(self.id_to_token):
                decoded_tokens.append(self.id_to_token[token_id])
            else:
                print(f"Предупреждение: ID токена {token_id} вне диапазона словаря.")
                decoded_tokens.append('') # Или можно использовать <unk>

        # Объединяем токены, убираем символ конца слова и заменяем пробелы
        decoded_text = "".join(decoded_tokens).replace('</w>', ' ').strip()
        return decoded_text.replace(' ', '_') # Возвращаем исходный формат с подчеркиваниями

# --- Пример использования ---
if __name__ == "__main__":
    corpus = "Ученик_учится_в_школе,_а_учитель_учит_ученика"

    # Создаем экземпляр BPE, указывая количество объединений
    # Можно также указать vocab_size=X для желаемого размера словаря
    bpe_model = BPE(num_merges=10) # Выполним 10 итераций объединения

    print("--- Обучение BPE модели ---")
    bpe_model.fit(corpus)
    print("\n--- Обучение завершено ---")
    print(f"Финальный словарь BPE (отсортированный): {sorted(list(bpe_model.vocabulary))}")
    print(f"Выполненные объединения: {bpe_model.merges}")

    # Тестирование токенизации
    print("\n--- Тестирование токенизации ---")
    text_to_encode = "Ученик_учится_в_школе,_а_учитель_учит_ученика"
    encoded_ids = bpe_model.encode(text_to_encode)
    print(f"Исходный текст: '{text_to_encode}'")
    print(f"Закодированные ID: {encoded_ids}")

    # Декодирование
    decoded_text = bpe_model.decode(encoded_ids)
    print(f"Декодированный текст: '{decoded_text}'")

    # Проверка на новое слово (OOV)
    print("\n--- Тестирование OOV слова ---")
    oov_text = "Учительница_учит"
    encoded_oov_ids = bpe_model.encode(oov_text)
    print(f"OOV текст: '{oov_text}'")
    print(f"Закодированные ID OOV: {encoded_oov_ids}")
    decoded_oov_text = bpe_model.decode(encoded_oov_ids)
    print(f"Декодированный OOV текст: '{decoded_oov_text}'")

    # Пример токенов по ID
    print("\n--- Токены по ID ---")
    for i, token in enumerate(bpe_model.id_to_token):
        print(f"ID {i}: '{token}'")

#WordPiece

In [ ]:
import collections

class WordPieceTokenizer:
    def __init__(self):
        self.vocab = set()
        self.merges = []

    def preprocess_text(self, text):
        """
        Предварительная обработка текста: приведение к нижнему регистру и разбиение на слова.
        """
        # Удаляем знаки препинания и приводим к нижнему регистру
        processed_text = text.lower().replace('...', '').replace('.', '')
        # Разбиваем на слова по пробелам
        words = processed_text.split(' ')
        # Добавляем пробелы как отдельные токены между словами
        initial_tokens = []
        for i, word in enumerate(words):
            if word: # Убедимся, что слово не пустое
                initial_tokens.extend(list(word))
            if i < len(words) - 1:
                initial_tokens.append(' ') # Добавляем пробел между словами
        return initial_tokens, words # Возвращаем начальные токены и список слов для удобства

    def calculate_frequencies(self, tokens):
        """
        Подсчет частот отдельных токенов (униграмм) и биграмм в текущем корпусе.
        """
        unigram_freq = collections.defaultdict(int)
        bigram_freq = collections.defaultdict(int)

        for i in range(len(tokens)):
            unigram_freq[tokens[i]] += 1
            if i < len(tokens) - 1:
                bigram_freq[(tokens[i], tokens[i+1])] += 1
        return unigram_freq, bigram_freq

    def calculate_score(self, unigram_freq, bigram_freq):
        """
        Вычисление оценки слияния для всех возможных биграмм.
        Score(A, B) = frequency(AB) / (frequency(A) * frequency(B))
        """
        scores = {}
        for (token_a, token_b), freq_ab in bigram_freq.items():
            freq_a = unigram_freq[token_a]
            freq_b = unigram_freq[token_b]
            if freq_a > 0 and freq_b > 0: # Избегаем деления на ноль
                score = freq_ab / (freq_a * freq_b)
                scores[(token_a, token_b)] = score
        return scores

    def merge_tokens(self, tokens, best_bigram):
        """
        Слияние лучшей биграммы в корпусе.
        """
        merged_token = "".join(best_bigram)
        new_tokens = []
        i = 0
        while i < len(tokens):
            if i + 1 < len(tokens) and (tokens[i], tokens[i+1]) == best_bigram:
                new_tokens.append(merged_token)
                i += 2 # Пропускаем оба слитых токена
            else:
                new_tokens.append(tokens[i])
                i += 1
        return new_tokens

    def train(self, text, target_vocab_size=None, num_merges=None):
        """
        Обучение WordPiece токенизатора.
        Итеративно сливает токены до достижения целевого размера словаря
        или заданного количества слияний.
        """
        print(f"Исходный текст: '{text}'")
        current_tokens, _ = self.preprocess_text(text)

        # Начальный словарь состоит из всех уникальных символов
        self.vocab = set(current_tokens)
        print(f"\nШаг 1: Начальный словарь (V0) содержит {len(self.vocab)} токенов.")
        print(f"Начальный корпус: {current_tokens}")

        merges_count = 0
        while True:
            unigram_freq, bigram_freq = self.calculate_frequencies(current_tokens)
            scores = self.calculate_score(unigram_freq, bigram_freq)

            if not scores:
                print("\nНет больше биграмм для слияния. Остановка.")
                break

            # Находим биграмму с наивысшей оценкой
            best_bigram = max(scores, key=scores.get)
            best_score = scores[best_bigram]

            merged_token = "".join(best_bigram)

            # Критерий остановки: если новый токен уже в словаре
            if merged_token in self.vocab:
                # Если лучший токен уже существует, удалим его из scores, чтобы найти следующий лучший
                del scores[best_bigram]
                continue # Продолжаем поиск

            # Выводим информацию о текущем слиянии
            print(f"\nИтерация {merges_count + 1}:")
            print(f"  Лучшая биграмма для слияния: '{best_bigram[0]}' + '{best_bigram[1]}' -> '{merged_token}' (Score: {best_score:.4f})")

            # Выполняем слияние в корпусе
            current_tokens = self.merge_tokens(current_tokens, best_bigram)

            # Добавляем новый токен в словарь и сохраняем слияние
            self.vocab.add(merged_token)
            self.merges.append(best_bigram)
            merges_count += 1

            print(f"  Корпус после слияния: {current_tokens}")
            print(f"  Текущий размер словаря: {len(self.vocab)}")

            # Критерии остановки
            if target_vocab_size is not None and len(self.vocab) >= target_vocab_size:
                print(f"\nДостигнут целевой размер словаря ({target_vocab_size}). Остановка.")
                break
            if num_merges is not None and merges_count >= num_merges:
                print(f"\nДостигнуто заданное количество слияний ({num_merges}). Остановка.")
                break

        print(f"\nОбучение завершено. Итоговый словарь содержит {len(self.vocab)} токенов.")
        return sorted(list(self.vocab))

    def tokenize(self, text):
        """
        Токенизация нового текста с использованием обученного словаря WordPiece.
        """
        if not hasattr(self, 'vocab') or len(self.vocab) == 0:
            raise ValueError("Токенизатор не обучен. Сначала вызовите метод train().")

        print(f"\n--- Токенизация нового текста ---")
        print(f"Текст для токенизации: '{text}'")

        # Предварительная обработка текста для токенизации
        processed_text = text.lower().replace('...', '').replace('.', '')
        words = processed_text.split(' ')

        final_tokens = []
        for word in words:
            if not word: # Пропускаем пустые строки, если они возникли из-за множественных пробелов
                continue

            word_tokens = []
            remaining_word = word

            while remaining_word:
                found_match = False
                # Ищем самый длинный подтокен из словаря, который является префиксом оставшейся части слова
                for i in range(len(remaining_word), 0, -1):
                    subword = remaining_word[:i]

                    if subword in self.vocab:
                        word_tokens.append(subword)
                        remaining_word = remaining_word[i:]
                        found_match = True
                        break

                if not found_match:
                    # Если не удалось найти соответствующий токен, разбиваем на символы
                    # или используем токен [UNK]
                    if remaining_word[0] in self.vocab: # Если символ есть в словаре
                        word_tokens.append(remaining_word[0])
                    else: # Если символ даже не в начальном словаре
                        word_tokens.append('[UNK]')
                    remaining_word = remaining_word[1:]

            final_tokens.extend(word_tokens)
            final_tokens.append(' ') # Добавляем пробел между токенизированными словами

        # Удаляем последний пробел, если он есть
        if final_tokens and final_tokens[-1] == ' ':
            final_tokens.pop()

        print(f"Токенизированный текст: {final_tokens}")
        return final_tokens

    def get_vocab(self):
        """Возвращает текущий словарь токенов."""
        return sorted(list(self.vocab))

    def get_merges(self):
        """Возвращает список выполненных слияний."""
        return self.merges

# --- Пример использования ---
if __name__ == "__main__":
    text_to_train = "В училище учитель учит ученикам по новому учебнику"

    # Создаем экземпляр токенизатора
    tokenizer = WordPieceTokenizer()

    # Обучаем токенизатор
    final_vocab = tokenizer.train(text_to_train, num_merges=50)

    print(f"\nИтоговый словарь WordPiece: {final_vocab}")

    # Токенизируем тот же текст
    tokenizer.tokenize(text_to_train)

    # Пример токенизации нового слова
    new_text = "учительница"
    # Добавим '##ница' в словарь для демонстрации
    if 'ница' not in final_vocab:
        tokenizer.vocab.add('ница')
        tokenizer.vocab.add('##ница')
        final_vocab = tokenizer.get_vocab()

    print(f"\nИтоговый словарь WordPiece (с 'ница' для демонстрации): {final_vocab}")
    tokenizer.tokenize(new_text)

    # Демонстрация с упрощенным словарем
    simplified_tokenizer = WordPieceTokenizer()
    simplified_tokenizer.vocab = set(['у', 'ч', 'и', 'т', 'е', 'л', 'ь', 'н', 'ц', 'а', 'учитель', '##ница', '##тель', '##а'])
    print(f"\n--- Демонстрация токенизации подслов с упрощенным словарем ---")
    simplified_tokenizer.tokenize("учительница")

# Униграмной языковой модели

In [ ]:
import collections
import re
import math

class UnigramLanguageModel:
    """
    Реализация Униграмной Языковой Модели.
    Эта модель предполагает, что вероятность каждого слова в последовательности
    не зависит от других слов.
    """
    def __init__(self):
        self.word_counts = collections.defaultdict(int) # Счетчик вхождений каждого слова
        self.total_words = 0 # Общее количество слов в корпусе
        self.vocabulary = set() # Множество уникальных слов (словарь)

    def _tokenize(self, text):
        """
        Приватный метод для токенизации текста.
        Приводит текст к нижнему регистру и разбивает на слова, удаляя знаки препинания.
        :param text: Входная строка текста.
        :return: Список токенов (слов).
        """
        # Приводим к нижнему регистру
        text = text.lower()
        # Удаляем знаки препинания (кроме тех, что могут быть частью слова, но для простоты здесь удаляем все)
        # Использование регулярного выражения для извлечения только буквенных последовательностей
        tokens = re.findall(r'\b[а-яё]+\b', text)
        return tokens

    def train(self, corpus_text):
        """
        Обучает униграмную языковую модель на заданном текстовом корпусе.
        Подсчитывает частоты слов и формирует словарь.
        :param corpus_text: Строка, представляющая обучающий корпус.
        """
        print("--- Начало обучения Униграмной Модели ---")
        tokens = self._tokenize(corpus_text)

        if not tokens:
            print("Предупреждение: Корпус пуст или не содержит слов после токенизации.")
            return

        self.total_words = len(tokens)

        for word in tokens:
            self.word_counts[word] += 1
            self.vocabulary.add(word)

        print(f"Обучение завершено. Общее количество слов (N): {self.total_words}")
        print(f"Размер словаря (|V|): {len(self.vocabulary)}")
        print(f"Частоты слов: {dict(self.word_counts)}")
        print("--- Обучение завершено ---")

    def get_word_probability(self, word, smoothing=None):
        """
        Возвращает вероятность слова P(w).
        :param word: Слово, для которого нужно рассчитать вероятность.
        :param smoothing: Метод сглаживания ('laplace' для сглаживания по Лапласу, None для MLE).
        :return: Вероятность слова.
        """
        # Если модель не обучена или корпус пуст
        if self.total_words == 0:
            print("Ошибка: Модель не обучена или корпус пуст. Невозможно рассчитать вероятность.")
            return 0.0

        count_w = self.word_counts[word] # Count(w)

        if smoothing == 'laplace':
            # Сглаживание по Лапласу: (Count(w) + 1) / (N + |V|)
            probability = (count_w + 1) / (self.total_words + len(self.vocabulary))
            # print(f"P_Laplace('{word}') = ({count_w} + 1) / ({self.total_words} + {len(self.vocabulary)}) = {probability:.4f}")
        else:
            # Метод максимального правдоподобия (MLE): Count(w) / N
            if count_w == 0:
                # print(f"P_MLE('{word}') = 0 (слово не найдено в корпусе)")
                return 0.0 # Если слово не найдено, вероятность 0 без сглаживания
            probability = count_w / self.total_words
            # print(f"P_MLE('{word}') = {count_w} / {self.total_words} = {probability:.4f}")

        return probability

    def get_sequence_probability(self, sequence_text, smoothing=None):
        """
        Возвращает вероятность последовательности слов P(W).
        :param sequence_text: Строка, представляющая последовательность слов.
        :param smoothing: Метод сглаживания ('laplace' для сглаживания по Лапласу, None для MLE).
        :return: Вероятность последовательности.
        """
        print(f"\n--- Расчет вероятности последовательности: '{sequence_text}' ---")
        tokens = self._tokenize(sequence_text)

        if not tokens:
            print("Предупреждение: Последовательность пуста или не содержит слов после токенизации.")
            return 0.0

        sequence_probability = 1.0
        probabilities_list = []

        for word in tokens:
            p_word = self.get_word_probability(word, smoothing)
            probabilities_list.append(p_word)
            sequence_probability *= p_word

            # Если хоть одно слово имеет нулевую вероятность без сглаживания,
            # вся последовательность будет иметь нулевую вероятность.
            if smoothing != 'laplace' and p_word == 0:
                print(f"Слово '{word}' имеет нулевую вероятность. Вероятность всей последовательности = 0.")
                return 0.0 # Оптимизация: если P(w)=0, то произведение будет 0

        print(f"Токены последовательности: {tokens}")
        print(f"Вероятности отдельных слов: {probabilities_list}")
        print(f"Итоговая вероятность последовательности: {sequence_probability}")
        print("--- Расчет завершен ---")
        return sequence_probability

# --- Пример использования ---
if __name__ == "__main__":
    corpus_example = "Ученики пишут диктант. Учитель диктует. Мы пишем, он пишет, они пишут. Пишите внимательно."

    # Создаем экземпляр модели
    unigram_model = UnigramLanguageModel()

    # Обучаем модель на корпусе
    unigram_model.train(corpus_example)

    print("\n--- Расчет вероятностей отдельных слов (MLE) ---")
    words_to_check = ['пишут', 'учитель', 'школа', 'мы']
    for word in words_to_check:
        prob = unigram_model.get_word_probability(word, smoothing=None)
        print(f"P_MLE('{word}') = {prob:.4f}")

    print("\n--- Расчет вероятностей отдельных слов (Лаплас) ---")
    for word in words_to_check:
        prob = unigram_model.get_word_probability(word, smoothing='laplace')
        print(f"P_Laplace('{word}') = {prob:.4f}")

    # Расчет вероятности предложения (без сглаживания)
    sentence_mle = "Ученики пишут диктант"
    prob_mle = unigram_model.get_sequence_probability(sentence_mle, smoothing=None)
    print(f"Вероятность предложения (MLE): {prob_mle:.10f}")

    sentence_mle_oov = "Ученики пишут школа" # Содержит OOV слово 'школа'
    prob_mle_oov = unigram_model.get_sequence_probability(sentence_mle_oov, smoothing=None)
    print(f"Вероятность предложения с OOV (MLE): {prob_mle_oov:.10f}")

    # Расчет вероятности предложения (со сглаживанием по Лапласу)
    sentence_laplace = "Ученики пишут диктант"
    prob_laplace = unigram_model.get_sequence_probability(sentence_laplace, smoothing='laplace')
    print(f"Вероятность предложения (Лаплас): {prob_laplace:.10f}")

    sentence_laplace_oov = "Ученики пишут школа" # Содержит OOV слово 'школа'
    prob_laplace_oov = unigram_model.get_sequence_probability(sentence_laplace_oov, smoothing='laplace')
    print(f"Вероятность предложения с OOV (Лаплас): {prob_laplace_oov:.10f}")

    # Пример из документации: "Ученики пишут диктант Учитель диктует Мы пишем он пишет они пишут Пишите внимательно"
    full_sentence = "Ученики пишут диктант Учитель диктует Мы пишем он пишет они пишут Пишите внимательно"
    prob_full_mle = unigram_model.get_sequence_probability(full_sentence, smoothing=None)
    print(f"Вероятность полного предложения (MLE): {prob_full_mle:.15f}")

    prob_full_laplace = unigram_model.get_sequence_probability(full_sentence, smoothing='laplace')
    print(f"Вероятность полного предложения (Лаплас): {prob_full_laplace:.15f}")


# SentencePiece

In [ ]:
import sentencepiece as spm
import os
import tempfile

class SentencePieceWrapper:
    """
    Класс-обертка для демонстрации использования библиотеки SentencePiece.
    Позволяет обучать модель, токенизировать и детокенизировать текст.
    """
    def __init__(self):
        self.sp_processor = spm.SentencePieceProcessor()
        self.model_prefix = None
        self.model_path = None
        self.vocab_path = None
        self._is_model_loaded = False # Добавляем флаг для отслеживания состояния загрузки модели

    def train(self, text_data, model_prefix="my_sp_model", vocab_size=8000, model_type='unigram', character_coverage=1.0):
        """
        Обучает модель SentencePiece на заданном текстовом корпусе.
        Создает временный файл для обучения и сохраняет модель.

        :param text_data: Строка, содержащая обучающий текст.
        :param model_prefix: Префикс для имен файлов модели (model.model и model.vocab).
        :param vocab_size: Желаемый размер словаря подслов.
        :param model_type: Алгоритм обучения ('unigram' или 'bpe').
        :param character_coverage: Процент символов, которые должны быть покрыты моделью.
        """
        print(f"--- Начинается обучение модели SentencePiece ---")
        print(f"Текст для обучения (фрагмент): '{text_data[:100]}...'")

        # Создаем временный файл для обучения
        # NamedTemporaryFile гарантирует уникальное имя и автоматическое удаление при закрытии
        with tempfile.NamedTemporaryFile(mode="w", encoding="utf-8", delete=False) as temp_file:
            temp_file.write(text_data)
            train_file_name = temp_file.name

        self.model_prefix = model_prefix
        self.model_path = f"{model_prefix}.model"
        self.vocab_path = f"{model_prefix}.vocab"

        try:
            print(f"Обучение с параметрами: vocab_size={vocab_size}, model_type='{model_type}'")
            spm.SentencePieceTrainer.train(
                input=train_file_name,
                model_prefix=self.model_prefix,
                vocab_size=vocab_size,
                character_coverage=character_coverage,
                model_type=model_type
            )
            print(f"Обучение завершено. Созданы файлы: '{self.model_path}' и '{self.vocab_path}'")
        except Exception as e:
            print(f"Ошибка при обучении модели SentencePiece: {e}")
            # Попытаемся удалить созданные файлы, если обучение не удалось
            self._cleanup_files()
            raise # Перевыбрасываем исключение
        finally:
            # Удаляем временный файл после обучения
            if os.path.exists(train_file_name):
                os.remove(train_file_name)
                print(f"Временный файл обучения удален: '{train_file_name}'")

        # Загружаем обученную модель
        self.load_model(self.model_path)
        print("--- Обучение и загрузка модели завершены ---")

    def load_model(self, model_path):
        """
        Загружает обученную модель SentencePiece из файла.
        :param model_path: Путь к файлу модели (.model).
        """
        print(f"--- Загрузка модели SentencePiece ---")
        try:
            self.sp_processor.load(model_path)
            self._is_model_loaded = True # Устанавливаем флаг после успешной загрузки
            print(f"Модель SentencePiece успешно загружена из: '{model_path}'")
        except Exception as e:
            self._is_model_loaded = False # Сбрасываем флаг в случае ошибки
            print(f"Ошибка при загрузке модели SentencePiece: {e}")
            raise # Перевыбрасываем исключение
        print("--- Загрузка модели завершена ---")

    def encode_as_ids(self, text):
        """
        Кодирует текст в последовательность числовых ID токенов.
        :param text: Входная строка текста.
        :return: Список числовых ID токенов.
        """
        if not self._is_model_loaded: # Используем наш флаг
            raise ValueError("Модель SentencePiece не загружена. Сначала обучите или загрузите модель.")

        print(f"\n--- Кодирование текста в ID ---")
        print(f"Исходный текст: '{text}'")
        encoded_ids = self.sp_processor.encode_as_ids(text)
        print(f"Закодированные ID: {encoded_ids}")
        return encoded_ids

    def encode_as_pieces(self, text):
        """
        Кодирует текст в последовательность строковых токенов (подслов).
        :param text: Входная строка текста.
        :return: Список строковых токенов.
        """
        if not self._is_model_loaded: # Используем наш флаг
            raise ValueError("Модель SentencePiece не загружена. Сначала обучите или загрузите модель.")

        print(f"\n--- Кодирование текста в подслова ---")
        print(f"Исходный текст: '{text}'")
        encoded_pieces = self.sp_processor.encode_as_pieces(text)
        print(f"Закодированные подслова: {encoded_pieces}")
        return encoded_pieces

    def decode_ids(self, ids):
        """
        Декодирует последовательность числовых ID токенов обратно в строку.
        :param ids: Список числовых ID токенов.
        :return: Декодированная строка.
        """
        if not self._is_model_loaded: # Используем наш флаг
            raise ValueError("Модель SentencePiece не загружена. Сначала обучите или загрузите модель.")

        print(f"\n--- Декодирование ID в текст ---")
        print(f"ID для декодирования: {ids}")
        decoded_text = self.sp_processor.decode_ids(ids)
        print(f"Декодированный текст: '{decoded_text}'")
        return decoded_text

    def decode_pieces(self, pieces):
        """
        Декодирует последовательность строковых токенов (подслов) обратно в строку.
        :param pieces: Список строковых токенов.
        :return: Декодированная строка.
        """
        if not self._is_model_loaded: # Используем наш флаг
            raise ValueError("Модель SentencePiece не загружена. Сначала обучите или загрузите модель.")

        print(f"\n--- Декодирование подслов в текст ---")
        print(f"Подслова для декодирования: {pieces}")
        decoded_text = self.sp_processor.decode_pieces(pieces)
        print(f"Декодированный текст: '{decoded_text}'")
        return decoded_text

    def _cleanup_files(self):
        """
        Удаляет файлы модели SentencePiece (.model и .vocab), если они существуют.
        """
        print(f"\n--- Очистка временных файлов модели ---")
        if self.model_path and os.path.exists(self.model_path):
            os.remove(self.model_path)
            print(f"Удален файл модели: '{self.model_path}'")
        if self.vocab_path and os.path.exists(self.vocab_path):
            os.remove(self.vocab_path)
            print(f"Удален файл словаря: '{self.vocab_path}'")

# --- Пример использования ---
if __name__ == "__main__":
    # Пример текста для обучения
    corpus_text = """Учитель говорит: «Откройте учебники». Ученики открывают учебники. Откройте страницу 15. Страница сложная. Страницы учебника содержат много информации."""

    # Создаем экземпляр класса SentencePieceWrapper
    sp_wrapper = SentencePieceWrapper()

    try:
        # Обучаем модель с заданным размером словаря (для демонстрации)
        sp_wrapper.train(corpus_text, vocab_size=50, model_type='unigram')

        # Тестируем кодирование и декодирование
        input_text_1 = "Ученики открывают учебники."
        encoded_ids_1 = sp_wrapper.encode_as_ids(input_text_1)
        decoded_text_1 = sp_wrapper.decode_ids(encoded_ids_1)
        print(f"\nПроверка обратимости 1: Исходный: '{input_text_1}', Декодированный: '{decoded_text_1}'")
        assert input_text_1 == decoded_text_1, "Ошибка обратимости для input_text_1"

        input_text_2 = "Страница сложная."
        encoded_pieces_2 = sp_wrapper.encode_as_pieces(input_text_2)
        decoded_text_2 = sp_wrapper.decode_pieces(encoded_pieces_2)
        print(f"\nПроверка обратимости 2: Исходный: '{input_text_2}', Декодированный: '{decoded_text_2}'")
        assert input_text_2 == decoded_text_2, "Ошибка обратимости для input_text_2"

        # Пример с OOV-словом (если оно не было полностью обучено)
        oov_text = "Преподавательница читает."
        print(f"\n--- Тестирование OOV-слова ---")
        encoded_oov_ids = sp_wrapper.encode_as_ids(oov_text)
        decoded_oov_text = sp_wrapper.decode_ids(encoded_oov_ids)
        print(f"Исходный OOV: '{oov_text}', Закодированные ID: {encoded_oov_ids}, Декодированный: '{decoded_oov_text}'")

    except Exception as e:
        print(f"Произошла ошибка в процессе выполнения: {e}")
    finally:
        # Очищаем созданные файлы модели
        sp_wrapper._cleanup_files()


# Класс для работы с Hugging Face Tokenizers






In [ ]:
import os
import tempfile
from tokenizers import Tokenizer, models, pre_tokenizers, normalizers, processors, trainers
from tokenizers.normalizers import Sequence, NFD, Lowercase, StripAccents
from typing import List, Optional, Dict, Any, Union

class HuggingFaceTokenizerWrapper:
    """
    Класс-обертка для работы с библиотекой Hugging Face Tokenizers.

    Предоставляет функциональность для обучения, сохранения, загрузки,
    кодирования и декодирования текста с использованием различных
    алгоритмов субтокенизации (BPE, WordPiece, Unigram).
    """

    def __init__(self):
        """
        Инициализирует обертку токенизатора.
        """
        self.tokenizer: Optional[Tokenizer] = None
        self.model_config: Dict[str, Any] = {}
        self.trained_model_path: Optional[str] = None
        print("HuggingFaceTokenizerWrapper: Инициализация.")

    def _create_temp_train_file(self, text_data: Union[str, List[str]]) -> str:
        """
        Создает временный файл(ы) для обучения токенизатора.

        :param text_data: Строка или список строк с обучающими данными.
        :return: Путь к временному файлу обучения.
        """
        # Используем NamedTemporaryFile для безопасного создания временных файлов
        # и их автоматического удаления при закрытии
        with tempfile.NamedTemporaryFile(mode="w", encoding="utf-8", delete=False) as temp_file:
            if isinstance(text_data, list):
                temp_file.write("\n".join(text_data))
            else:
                temp_file.write(text_data)
            temp_file_path = temp_file.name
        print(f"HuggingFaceTokenizerWrapper: Создан временный файл для обучения: '{temp_file_path}'")
        return temp_file_path

    def train(self,
              corpus_text: Union[str, List[str]],
              model_type: str = 'bpe',
              vocab_size: int = 30000,
              min_frequency: int = 2,
              special_tokens: Optional[List[str]] = None,
              output_path: str = "my_hf_tokenizer.json",
              normalizer_config: Optional[Dict[str, Any]] = None,
              pretokenizer_config: Optional[Dict[str, Any]] = None,
              postprocessor_config: Optional[Dict[str, Any]] = None):
        """
        Обучает новый токенизатор на заданном корпусе.

        :param corpus_text: Текст или список текстов для обучения.
        :param model_type: Тип модели токенизации ('bpe', 'wordpiece', 'unigram', 'wordlevel', 'charlevel').
                           По умолчанию 'bpe'.
        :param vocab_size: Желаемый размер словаря.
        :param min_frequency: Минимальная частота токена для включения в словарь.
        :param special_tokens: Список специальных токенов (например, [UNK], [CLS], [SEP]).
                               По умолчанию: ["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"].
        :param output_path: Путь для сохранения обученного токенизатора в формате JSON.
        :param normalizer_config: Словарь для настройки нормализатора.
                                  Пример: {'type': 'Sequence', 'sub_normalizers': [{'type': 'NFD'}, {'type': 'Lowercase'}]}
        :param pretokenizer_config: Словарь для настройки предварительного токенизатора.
                                   Пример: {'type': 'Whitespace'}
        :param postprocessor_config: Словарь для настройки постобработчика.
                                    Пример: {'type': 'BertProcessing'}
        """
        print(f"\n--- Начинается обучение токенизатора типа '{model_type}' ---")

        if special_tokens is None:
            special_tokens = ["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"]

        # 1. Инициализация модели токенизации
        if model_type == 'bpe':
            self.tokenizer = Tokenizer(models.BPE(unk_token="[UNK]"))
            trainer = trainers.BpeTrainer(vocab_size=vocab_size, min_frequency=min_frequency, special_tokens=special_tokens)
        elif model_type == 'wordpiece':
            self.tokenizer = Tokenizer(models.WordPiece(unk_token="[UNK]"))
            trainer = trainers.WordPieceTrainer(vocab_size=vocab_size, min_frequency=min_frequency, special_tokens=special_tokens)
        elif model_type == 'unigram':
            self.tokenizer = Tokenizer(models.Unigram())
            # UnigramTrainer не имеет min_frequency, но имеет limit_alphabet
            trainer = trainers.UnigramTrainer(vocab_size=vocab_size, special_tokens=special_tokens)
        elif model_type == 'wordlevel':
            self.tokenizer = Tokenizer(models.WordLevel(unk_token="[UNK]"))
            trainer = trainers.WordLevelTrainer(vocab_size=vocab_size, min_frequency=min_frequency, special_tokens=special_tokens)
        elif model_type == 'charlevel':
            self.tokenizer = Tokenizer(models.CharLevel(unk_token="[UNK]"))
            trainer = trainers.CharLevelTrainer(vocab_size=vocab_size, min_frequency=min_frequency, special_tokens=special_tokens)
        else:
            raise ValueError(f"Неизвестный тип модели: {model_type}. Поддерживаются: bpe, wordpiece, unigram, wordlevel, charlevel.")
        print(f"HuggingFaceTokenizerWrapper: Инициализирована модель: {model_type}")

        # 2. Настройка нормализатора
        if normalizer_config:
            self._set_normalizer(normalizer_config)
        else:
            # Нормализатор по умолчанию: NFD, Lowercase, StripAccents
            self.tokenizer.normalizer = Sequence([
                NFD(),
                Lowercase(),
                StripAccents()
            ])
            print("HuggingFaceTokenizerWrapper: Настроен нормализатор по умолчанию (NFD, Lowercase, StripAccents).")

        # 3. Настройка предварительного токенизатора
        if pretokenizer_config:
            self._set_pretokenizer(pretokenizer_config)
        else:
            # Предварительный токенизатор по умолчанию: Whitespace
            self.tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()
            print("HuggingFaceTokenizerWrapper: Настроен предварительный токенизатор по умолчанию (Whitespace).")

        # 4. Обучение токенизатора
        temp_file_path = self._create_temp_train_file(corpus_text)
        try:
            print(f"HuggingFaceTokenizerWrapper: Начинается обучение модели...")
            # Передаем объект trainer в метод train
            self.tokenizer.train([temp_file_path], trainer=trainer)
            print("HuggingFaceTokenizerWrapper: Обучение завершено.")
        except Exception as e:
            print(f"HuggingFaceTokenizerWrapper: Ошибка при обучении токенизатора: {e}")
            raise
        finally:
            if os.path.exists(temp_file_path):
                os.remove(temp_file_path)
                print(f"HuggingFaceTokenizerWrapper: Временный файл обучения удален: '{temp_file_path}'")

        # 5. Настройка постобработчика
        if postprocessor_config:
            self._set_postprocessor(postprocessor_config)
        else:
            # Постобработчик по умолчанию для BERT-подобных моделей
            if "[CLS]" in special_tokens and "[SEP]" in special_tokens:
                self.tokenizer.post_processor = processors.TemplateProcessing(
                    single="[CLS] $A [SEP]",
                    pair="[CLS] $A [SEP] $B:1 [SEP]:1",
                    special_tokens=[
                        ("[CLS]", self.tokenizer.token_to_id("[CLS]")),
                        ("[SEP]", self.tokenizer.token_to_id("[SEP]")),
                    ],
                )
                print("HuggingFaceTokenizerWrapper: Настроен постобработчик по умолчанию (BERT-совместимый).")
            else:
                print("HuggingFaceTokenizerWrapper: Постобработчик не настроен (специальные токены [CLS]/[SEP] отсутствуют).")


        # 6. Сохранение обученного токенизатора
        self.save_tokenizer(output_path)
        print(f"--- Обучение и сохранение токенизатора завершены ---")

    def _set_normalizer(self, config: Dict[str, Any]):
        """Настраивает нормализатор токенизатора."""
        norm_type = config.get('type')
        if norm_type == 'Sequence':
            sub_normalizers = []
            for sub_norm_cfg in config.get('sub_normalizers', []):
                sub_norm_type = sub_norm_cfg.get('type')
                if sub_norm_type == 'NFD': sub_normalizers.append(NFD())
                elif sub_norm_type == 'Lowercase': sub_normalizers.append(Lowercase())
                elif sub_norm_type == 'StripAccents': sub_normalizers.append(StripAccents())
                # Добавьте другие нормализаторы по мере необходимости
                else: print(f"HuggingFaceTokenizerWrapper: Неизвестный суб-нормализатор: {sub_norm_type}")
            self.tokenizer.normalizer = Sequence(sub_normalizers)
        elif norm_type == 'Lowercase':
            self.tokenizer.normalizer = Lowercase()
        # Добавьте другие типы нормализаторов по мере необходимости
        else:
            print(f"HuggingFaceTokenizerWrapper: Неизвестный тип нормализатора: {norm_type}. Используется по умолчанию.")
            self.tokenizer.normalizer = Sequence([NFD(), Lowercase(), StripAccents()])
        print(f"HuggingFaceTokenizerWrapper: Настроен нормализатор: {norm_type}")

    def _set_pretokenizer(self, config: Dict[str, Any]):
        """Настраивает предварительный токенизатор."""
        pre_tok_type = config.get('type')
        if pre_tok_type == 'Whitespace':
            self.tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()
        elif pre_tok_type == 'WhitespaceSplit':
            self.tokenizer.pre_tokenizer = pre_tokenizers.WhitespaceSplit()
        elif pre_tok_type == 'Punctuation':
            self.tokenizer.pre_tokenizer = pre_tokenizers.Punctuation()
        # Добавьте другие типы предварительных токенизаторов по мере необходимости
        else:
            print(f"HuggingFaceTokenizerWrapper: Неизвестный тип предварительного токенизатора: {pre_tok_type}. Используется по умолчанию.")
            self.tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()
        print(f"HuggingFaceTokenizerWrapper: Настроен предварительный токенизатор: {pre_tok_type}")

    def _set_postprocessor(self, config: Dict[str, Any]):
        """Настраивает постобработчик."""
        post_proc_type = config.get('type')
        if post_proc_type == 'TemplateProcessing':
            special_tokens_map = []
            for token_name, token_id in config.get('special_tokens', []):
                special_tokens_map.append((token_name, self.tokenizer.token_to_id(token_name)))
            self.tokenizer.post_processor = processors.TemplateProcessing(
                single=config.get('single'),
                pair=config.get('pair'),
                special_tokens=special_tokens_map
            )
        elif post_proc_type == 'BertProcessing':
            # Предполагаем, что CLS и SEP токены уже в словаре
            cls_id = self.tokenizer.token_to_id("[CLS]")
            sep_id = self.tokenizer.token_to_id("[SEP]")
            if cls_id is not None and sep_id is not None:
                self.tokenizer.post_processor = processors.BertProcessing(
                    sep=(f"[SEP]", sep_id),
                    cls=(f"[CLS]", cls_id)
                )
            else:
                print("HuggingFaceTokenizerWrapper: Не удалось настроить BertProcessing: отсутствуют [CLS] или [SEP] ID.")
        # Добавьте другие типы постобработчиков по мере необходимости
        else:
            print(f"HuggingFaceTokenizerWrapper: Неизвестный тип постобработчика: {post_proc_type}. Постобработчик не настроен.")
        print(f"HuggingFaceTokenizerWrapper: Настроен постобработчик: {post_proc_type}")


    def save_tokenizer(self, path: str):
        """
        Сохраняет обученный токенизатор в файл JSON.

        :param path: Путь к файлу для сохранения.
        """
        if self.tokenizer is None:
            raise ValueError("Токенизатор не обучен. Сначала вызовите метод 'train'.")
        try:
            self.tokenizer.save(path)
            self.trained_model_path = path
            print(f"HuggingFaceTokenizerWrapper: Токенизатор успешно сохранен в '{path}'")
        except Exception as e:
            print(f"HuggingFaceTokenizerWrapper: Ошибка при сохранении токенизатора: {e}")
            raise

    def load_tokenizer(self, path: str):
        """
        Загружает токенизатор из файла JSON.

        :param path: Путь к файлу JSON с токенизатором.
        """
        print(f"\n--- Загрузка токенизатора из '{path}' ---")
        try:
            self.tokenizer = Tokenizer.from_file(path)
            self.trained_model_path = path
            print(f"HuggingFaceTokenizerWrapper: Токенизатор успешно загружен из '{path}'")
        except Exception as e:
            print(f"HuggingFaceTokenizerWrapper: Ошибка при загрузке токенизатора: {e}")
            raise
        print(f"--- Загрузка токенизатора завершена ---")


    def encode(self, text: str, add_special_tokens: bool = True) -> Dict[str, Any]:
        """
        Кодирует входной текст.

        :param text: Текст для кодирования.
        :param add_special_tokens: Добавлять ли специальные токены (например, [CLS], [SEP]).
        :return: Словарь с закодированными данными (ids, tokens, attention_mask, type_ids, offsets).
        """
        if self.tokenizer is None:
            raise ValueError("Токенизатор не загружен или не обучен. Сначала обучите или загрузите модель.")

        print(f"\n--- Кодирование текста: '{text}' ---")
        # Метод encode возвращает объект Encoding, который содержит все необходимые данные
        encoding = self.tokenizer.encode(text, add_special_tokens=add_special_tokens)

        result = {
            # 'normalized_str' не является прямым атрибутом Encoding объекта.
            # Если нужна нормализованная строка, ее нужно получить до кодирования.
            # Для простоты, мы будем использовать оригинальную строку, если она нужна для вывода.
            # "normalized_str": encoding.normalized_str, # Эту строку убираем
            "tokens": encoding.tokens,
            "ids": encoding.ids,
            "attention_mask": encoding.attention_mask,
            "type_ids": encoding.type_ids,
            "offsets": encoding.offsets
        }
        print(f"HuggingFaceTokenizerWrapper: Закодированные токены: {result['tokens']}")
        print(f"HuggingFaceTokenizerWrapper: Закодированные ID: {result['ids']}")
        return result

    def decode(self, ids: List[int], skip_special_tokens: bool = True) -> str:
        """
        Декодирует последовательность ID токенов обратно в строку.

        :param ids: Список числовых ID токенов.
        :param skip_special_tokens: Пропускать ли специальные токены при декодировании.
        :return: Декодированная строка.
        """
        if self.tokenizer is None:
            raise ValueError("Токенизатор не загружен или не обучен. Сначала обучите или загрузите модель.")

        print(f"\n--- Декодирование ID: {ids} ---")
        decoded_text = self.tokenizer.decode(ids, skip_special_tokens=skip_special_tokens)
        print(f"HuggingFaceTokenizerWrapper: Декодированный текст: '{decoded_text}'")
        return decoded_text

    def get_vocab(self) -> Dict[str, int]:
        """
        Возвращает словарь токенов (токен -> ID).
        """
        if self.tokenizer is None:
            raise ValueError("Токенизатор не загружен или не обучен.")
        return self.tokenizer.get_vocab()

    def get_vocab_size(self) -> int:
        """
        Возвращает размер словаря токенизатора.
        """
        if self.tokenizer is None:
            raise ValueError("Токенизатор не загружен или не обучен.")
        return self.tokenizer.get_vocab_size()

    def _cleanup_files(self):
        """
        Удаляет файлы обученной модели токенизатора, если они существуют.
        """
        if self.trained_model_path and os.path.exists(self.trained_model_path):
            try:
                os.remove(self.trained_model_path)
                print(f"HuggingFaceTokenizerWrapper: Удален файл модели: '{self.trained_model_path}'")
            except OSError as e:
                print(f"HuggingFaceTokenizerWrapper: Ошибка при удалении файла '{self.trained_model_path}': {e}")
        # Также удаляем файл .vocab, если он создается (для BPE и WordPiece)
        vocab_path = self.trained_model_path.replace(".json", "-vocab.txt") if self.trained_model_path else None
        if vocab_path and os.path.exists(vocab_path):
            try:
                os.remove(vocab_path)
                print(f"HuggingFaceTokenizerWrapper: Удален файл словаря: '{vocab_path}'")
            except OSError as e:
                print(f"HuggingFaceTokenizerWrapper: Ошибка при удалении файла '{vocab_path}': {e}")


# --- Пример использования класса ---
if __name__ == "__main__":
    corpus_for_training = """
    В современном мире обработки естественного языка (НЛП) эффективная и быстрая токенизация
    является краеугольным камнем для работы с большими языковыми моделями.
    Библиотека Hugging Face Tokenizers — это высокопроизводительная реализация самых популярных
    алгоритмов токенизации, разработанная для обеспечения скорости, гибкости и совместимости
    с экосистемой Hugging Face Transformers.
    """

    # Инициализация обертки
    hf_tokenizer = HuggingFaceTokenizerWrapper()

    # Путь для сохранения модели
    output_model_file = "my_custom_hf_tokenizer.json"

    try:
        # Обучение BPE токенизатора
        hf_tokenizer.train(
            corpus_for_training,
            model_type='bpe',
            vocab_size=100, # Небольшой словарь для демонстрации
            min_frequency=1,
            special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"],
            output_path=output_model_file
        )

        # Загрузка обученного токенизатора
        loaded_hf_tokenizer = HuggingFaceTokenizerWrapper()
        loaded_hf_tokenizer.load_tokenizer(output_model_file)

        # Кодирование текста
        text_to_process = "Hugging Face Tokenizers - это мощный инструмент для НЛП."
        encoded_output = loaded_hf_tokenizer.encode(text_to_process)

        print("\n--- Детали закодированного вывода ---")
        # print(f"Нормализованная строка: {encoded_output['normalized_str']}") # Эта строка удалена
        print(f"Токены: {encoded_output['tokens']}")
        print(f"ID токенов: {encoded_output['ids']}")
        print(f"Маска внимания: {encoded_output['attention_mask']}")
        print(f"Типы токенов: {encoded_output['type_ids']}")
        print(f"Офсеты: {encoded_output['offsets']}")

        # Декодирование текста
        decoded_text = loaded_hf_tokenizer.decode(encoded_output['ids'])
        print(f"\nПроверка обратимости: Исходный: '{text_to_process}', Декодированный: '{decoded_text}'")
        # Примечание: Декодированный текст может отличаться от исходного из-за нормализации и токенизации.
        # Например, пунктуация может быть отделена, регистр изменен.

        # Пример с другим типом модели (WordPiece)
        print("\n--- Демонстрация WordPiece токенизатора ---")
        wordpiece_tokenizer = HuggingFaceTokenizerWrapper()
        wordpiece_tokenizer.train(
            corpus_for_training,
            model_type='wordpiece',
            vocab_size=100,
            min_frequency=1,
            special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"],
            output_path="my_wordpiece_tokenizer.json"
        )
        wp_encoded = wordpiece_tokenizer.encode("Hugging Face Tokenizers - это мощный инструмент для НЛП.")
        wp_decoded = wordpiece_tokenizer.decode(wp_encoded['ids'])
        print(f"WordPiece Декодированный: '{wp_decoded}'")

    except Exception as e:
        print(f"Произошла ошибка в процессе выполнения: {e}")
    finally:
        # Очистка созданных файлов
        hf_tokenizer._cleanup_files()
        if os.path.exists("my_wordpiece_tokenizer.json"):
            os.remove("my_wordpiece_tokenizer.json")
            print("HuggingFaceTokenizerWrapper: Удален файл модели: 'my_wordpiece_tokenizer.json'")
        # Также удаляем файл .vocab, если он создается для WordPiece
        if os.path.exists("my_wordpiece_tokenizer-vocab.txt"):
            os.remove("my_wordpiece_tokenizer-vocab.txt")
            print("HuggingFaceTokenizerWrapper: Удален файл словаря: 'my_wordpiece_tokenizer-vocab.txt'")
